In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
html=urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup=BeautifulSoup(html,'html.parser')
# print(soup.prettify())

### 1. Extract the data from the html files ###
* Append all the cell into one list
* Pull out PostalCode, Borough, Neighborhood according to the interval and save to the list
* Transform the list into dataframe

In [2]:
df=pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
children=[]
for child in soup.table.find_all('td'):
    children.append(child.text)
PostalCode=[children[i] for i in range(0,len(children),3)]
Borough=[children[i] for i in range(1,len(children),3)]
Neighborhood=[children[i].split('\n')[0] for i in range(2,len(children),3)]
df['PostalCode']=PostalCode
df['Borough']=Borough
df['Neighborhood']=Neighborhood
df=df.replace(to_replace='\n',value='')
df.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### 2. Delete the "Not Assigned" in Borough
* Get the logical value whether the cell contains "Not assigned"
* Apply the logical value to reserve the 'True' data

In [4]:
df=df[~df.Borough.str.contains("Not assigned")]
df.reset_index(drop=True,inplace=True)
df.head(7)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned


### 3. Combine the Neighborhood which share the same PostalCode
* Use the groupby to combine the Neighborhood

In [14]:
def f(x):
    return pd.Series(dict(Neighborhood='%s'%','.join(x['Neighborhood'])))

df=df.groupby(['PostalCode','Borough']).apply(f)
df.reset_index(inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### 4. Assign the Borough value to Neighborhood which is 'Not assigned' 
* Use apply to excute a if statement in every row to get the new 'Neighbohood' column

In [15]:
# df.
def reasign(x):
#     print(type(x))
    if x.Neighborhood=='Not assigned': 
        return x.Borough
    else:
        return x.Neighborhood

df['Neighborhood']=df.apply(reasign,axis=1)

In [16]:
df.shape

(103, 3)

In [13]:
A=[1,2,3]
B=['a','b','c']
dict('A')

ValueError: dictionary update sequence element #0 has length 1; 2 is required